In [19]:
from nordpool import elspot
import pandas as pd
import numpy as np
import openmeteo_requests
import requests_cache



### Custom functions

In [20]:
import sys
sys.path.append("..")

# Import the module and reload it every time
import importlib
import fun.api_f
importlib.reload(fun.api_f)

from fun.api_f import fetch_nordpool_data, get_weather_forecast


In [21]:
df = fetch_nordpool_data(area="EE")
df.head(5)

,start,end,value
0,2025-01-10 00:00:00+01:00,2025-01-10 01:00:00+01:00,79.76
1,2025-01-10 01:00:00+01:00,2025-01-10 02:00:00+01:00,79.74
2,2025-01-10 02:00:00+01:00,2025-01-10 03:00:00+01:00,54.95
3,2025-01-10 03:00:00+01:00,2025-01-10 04:00:00+01:00,50.94
4,2025-01-10 04:00:00+01:00,2025-01-10 05:00:00+01:00,66.18


### Get weather data

In [22]:
# Coordinates for Tallinn, Estonia
latitude = 59.437
longitude = 24.7535

df_weather_forecast = get_weather_forecast(latitude, longitude)
df_weather_forecast.head(5)

,time,temperature_2m,wind_speed_10m,direct_radiation,diffuse_radiation
0,2025-01-09 00:00:00+01:00,1.2,23.4,0.0,0.0
1,2025-01-09 01:00:00+01:00,1.3,25.6,0.0,0.0
2,2025-01-09 02:00:00+01:00,1.0,26.3,0.0,0.0
3,2025-01-09 03:00:00+01:00,1.3,27.0,0.0,0.0
4,2025-01-09 04:00:00+01:00,1.4,24.5,0.0,0.0
